In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import pickle
%matplotlib inline

df = pd.read_csv('sac_animals.csv', parse_dates=True)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21953 entries, 0 to 37350
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   objectid              21953 non-null  int64 
 1   animal_id             21953 non-null  object
 2   animal_name           16656 non-null  object
 3   animal_type           21953 non-null  object
 4   activity_number       6714 non-null   object
 5   intake_type           21953 non-null  object
 6   picked_up_location    18322 non-null  object
 7   intake_date           21953 non-null  object
 8   outcome_type          21953 non-null  object
 9   outcome_date          21953 non-null  object
 10  animal_age            21953 non-null  object
 11  animal_primarycolor   21953 non-null  object
 12  animal_sex            21953 non-null  object
 13  outcome_subtype       21861 non-null  object
 14  animal_sex_clean      21953 non-null  object
 15  animal_intake_status  21953 non-null

In [12]:
df.outcome_type.value_counts()
df = df[df['outcome_type'].isin(['ADOPTION' ,'FOSTER', 'RTO', 'EUTH', 'TRANSFER', 'RTF', 'DIED', 'EUTH VET'])]
df = df[df['days_spent'] > 0]

In [13]:
c_data = df[['animal_type', 'animal_sex_clean', 'animal_intake_status', 'name_unknown', 'has_activity_number',
            'animal_age_in_days', 'intake_type', 'color_clean', 'intake_times', 'days_spent']]
c_data = c_data.dropna()

x = c_data.drop('days_spent', axis=1)
x = x.apply(LabelEncoder().fit_transform)
y = c_data['days_spent']

In [14]:
xgb_final = pickle.load(open('sac_animals_predict_days.pkl', 'rb'))

In [15]:
preds = xgb_final.predict(x)

In [16]:
preds

array([ 7.7125597, 12.829391 , 13.596717 , ...,  4.1200323, 16.798197 ,
       16.745125 ], dtype=float32)

In [23]:
c_data

,animal_type,animal_sex_clean,animal_intake_status,name_unknown,has_activity_number,animal_age_in_days,intake_type,color_clean,intake_times,days_spent
0,DOG,Female,neutured,0,0,5110,STRAY,black,1,2
1,DOG,Female,neutured,0,0,3285,STRAY,other_solid,1,13
3,DOG,Male,neutured,0,0,2920,STRAY,other_solid,1,1
4,DOG,Male,intact,0,0,3285,STRAY,black,1,2
5,DOG,Male,neutured,0,1,3650,STRAY,orange,2,2
...,...,...,...,...,...,...,...,...,...,...
37346,CAT,Male,neutured,0,0,730,STRAY,tabby,1,15
37347,CAT,Male,neutured,0,0,2920,STRAY,black,1,6
37348,DOG,Male,intact,1,0,730,STRAY,black,1,12
37349,CAT,Female,neutured,0,0,730,STRAY,tabby,1,18


In [27]:
df['predicted_days_in_shelter'] = preds

In [29]:
df.to_csv('sacramento_full_data_with_predictions.csv')